# 뉴스 카테고리 분류 모델 (딥러닝)
* https://wikidocs.net/22933

뉴스 추천 모델을 만들기 위해서 사용자 로그 데이터와 사용자들에게 추천해 줄 뉴스 데이터가 필요합니다. 하지만 뉴스 데이터 중 분류가 제대로 되지 않은 데이터가 절반 가량 됩니다. 이러한 이유에서 뉴스 분류 모델을 만들어 뉴스 데이터를 카테고리에 맞게 분류하고자 한다. 또한 뉴스 추천 모델을 만들어 사용자들의 취향에 맞추어 뉴스를 서비스 하고 장기적으로 사용자를 늘릴 수 있게 하려한다.

In [1]:
import os
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

## 데이터 준비 및 전처리

뉴스데이터를 전처리하여 분류 및 추천 모델에 사용할 수 있게 합니다.

In [2]:
def new_df(p):
    name = []
    by = []
    date = []
    main = []
    content = []
    subject = []
    hangul = re.compile('[^ 가-힣]+')
    for (path, dir, files) in os.walk(p):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            title = os.path.splitext(filename)[0]
            if ext == '.xml':
                name.append(title[9:])
                by.append(title[:8])
                date.append(title[9:13])
                with open('%s/%s' % (path, filename), 'r') as f:
                    soup = BeautifulSoup(f, 'html.parser')
                    dataheadline = soup.find('headline')
                    head = dataheadline.get_text()
                    heads = hangul.sub('', head)
                    main.append(heads)
                    datacontent = soup.find('datacontent')
                    text = datacontent.get_text()
                    texts = hangul.sub('', text)
                    content.append(texts)
                    datasubject = soup.find(formalname = 'SubjectInfo')
                    ns = datasubject.attrs['value']
                    if ns == '':
                        datasubject = soup.find(formalname = 'AutoSubjectInfo')
                        ns = datasubject.attrs['value']
                    subject.append(ns[:3])
    return name, by, date, main, content, subject

In [3]:
%%time
a,b,c,d,e,f = new_df(os.getcwd())

CPU times: user 1h 11min 29s, sys: 2min 3s, total: 1h 13min 32s
Wall time: 1h 19min 9s


In [4]:
%%time
g = []
h = []
df = pd.DataFrame(a, columns = ['ID'])
df['Company'] = b
df['Year'] = c
df['Headline'] = d
df['Content'] = e
for i in range(len(df)):
    j = df['Headline'][i] + df['Content'][i]
    g.append(j)
df['News'] = g
for i in f:
    k = re.sub('[^A-Za-z0-9가-힣]', '', i)
    h.append(k)
df['Subject'] = h

CPU times: user 7.51 s, sys: 344 ms, total: 7.86 s
Wall time: 7.86 s


In [5]:
df.head()

,ID,Company,Year,Headline,Content,News,Subject
0,20191103170130003,07100501,2019,내년 커넥티드카 기술방식 확정과기정통부국토부 기술분과회의 가동,과학기술정보통신부와 국토교통부가 년 커넥티드카 통신 기술을 확정한다 주요 기술 방식...,내년 커넥티드카 기술방식 확정과기정통부국토부 기술분과회의 가동과학기술정보통신부와 ...,통신
1,20191103185930001,07100501,2019,취미플랫폼 덕업덧컴 김지훈 트레이너 챌린저 프로젝트 오픈,분야별 전문가와 함께하는 취미플랫폼 덕업닷컴이 연예인 전담 트레이너로 잘 알려진 김...,취미플랫폼 덕업덧컴 김지훈 트레이너 챌린저 프로젝트 오픈분야별 전문가와 함께하는 ...,이코노
2,20191103124208001,07100501,2019,제주삼다수 소외 아동 여명 초청 희망의 직업 체험 진행,제주삼다수가 직업 체험 기회를 제공하는 등 어린이의 꿈을 응원하는데 앞장서고 있다제...,제주삼다수 소외 아동 여명 초청 희망의 직업 체험 진행제주삼다수가 직업 체험 기회를...,전자
3,20191103130204001,07100501,2019,동아오츠카 자랑스러운 청소년 대상서 포카리스웨트 장학금 수여,동아오츠카대표 양동영 사장가 일 한국스카우트연맹회관 층 스카우트홀에서 개최한 제회 ...,동아오츠카 자랑스러운 청소년 대상서 포카리스웨트 장학금 수여동아오츠카대표 양동영 사...,전자
4,20191103090136001,07100501,2019,기획삼성 초대형로 프리미엄 시장서도 초격차,삼성전자는 글로벌 시장에서 년부터 년 연속 위를 하고 있다최근 발표한 마킷에 따...,기획삼성 초대형로 프리미엄 시장서도 초격차삼성전자는 글로벌 시장에서 년부터 년...,전자


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824874 entries, 0 to 824873
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        824874 non-null  object
 1   Company   824874 non-null  object
 2   Year      824874 non-null  object
 3   Headline  824874 non-null  object
 4   Content   824874 non-null  object
 5   News      824874 non-null  object
 6   Subject   824874 non-null  object
dtypes: object(7)
memory usage: 44.1+ MB


In [7]:
print('경제 :', len(df[df.Subject == '경제']))
print('사회 :', len(df[df.Subject == '사회']))
print('정치 :', len(df[df.Subject == '정치']))
print('문화 :', len(df[df.Subject == '문화']))
print('국제 :', len(df[df.Subject == '국제']))
print('IT :', len(df[df.Subject == 'IT']))
print('스포츠 :', len(df[df.Subject == '스포츠']))

경제 : 188713
사회 : 140406
정치 : 72950
문화 : 32703
국제 : 62004
IT : 18822
스포츠 : 17645


In [8]:
%%time
df.to_csv('news_df.csv', index = False)

CPU times: user 31.2 s, sys: 1.99 s, total: 33.1 s
Wall time: 33.4 s


In [9]:
%%time
data = pd.read_csv('news_df.csv')
data.head()

CPU times: user 27.7 s, sys: 4.88 s, total: 32.6 s
Wall time: 32.7 s


,ID,Company,Year,Headline,Content,News,Subject
0,20191103170130003,7100501,2019,내년 커넥티드카 기술방식 확정과기정통부국토부 기술분과회의 가동,과학기술정보통신부와 국토교통부가 년 커넥티드카 통신 기술을 확정한다 주요 기술 방식...,내년 커넥티드카 기술방식 확정과기정통부국토부 기술분과회의 가동과학기술정보통신부와 ...,통신
1,20191103185930001,7100501,2019,취미플랫폼 덕업덧컴 김지훈 트레이너 챌린저 프로젝트 오픈,분야별 전문가와 함께하는 취미플랫폼 덕업닷컴이 연예인 전담 트레이너로 잘 알려진 김...,취미플랫폼 덕업덧컴 김지훈 트레이너 챌린저 프로젝트 오픈분야별 전문가와 함께하는 ...,이코노
2,20191103124208001,7100501,2019,제주삼다수 소외 아동 여명 초청 희망의 직업 체험 진행,제주삼다수가 직업 체험 기회를 제공하는 등 어린이의 꿈을 응원하는데 앞장서고 있다제...,제주삼다수 소외 아동 여명 초청 희망의 직업 체험 진행제주삼다수가 직업 체험 기회를...,전자
3,20191103130204001,7100501,2019,동아오츠카 자랑스러운 청소년 대상서 포카리스웨트 장학금 수여,동아오츠카대표 양동영 사장가 일 한국스카우트연맹회관 층 스카우트홀에서 개최한 제회 ...,동아오츠카 자랑스러운 청소년 대상서 포카리스웨트 장학금 수여동아오츠카대표 양동영 사...,전자
4,20191103090136001,7100501,2019,기획삼성 초대형로 프리미엄 시장서도 초격차,삼성전자는 글로벌 시장에서 년부터 년 연속 위를 하고 있다최근 발표한 마킷에 따...,기획삼성 초대형로 프리미엄 시장서도 초격차삼성전자는 글로벌 시장에서 년부터 년...,전자


In [10]:
%%time
news_df = df.loc[(df.Company == '02100201')]
news_df['Subject'].value_counts()

CPU times: user 188 ms, sys: 31.5 ms, total: 220 ms
Wall time: 218 ms


경제     55412
사회     27513
정치     25530
미분류    18355
문화     17849
국제     17497
IT     15889
지역     13841
스포츠     1527
Name: Subject, dtype: int64

In [11]:
%%time
n1 = news_df[news_df['Subject'] == '미분류'].index
news_df = news_df.drop(n1)
n2 = news_df[news_df['Subject'] == '지역'].index
news_df = news_df.drop(n2)

# 경제 0, 사회 1, 정치 2, 문화 3, 국제 4, IT 5, 스포츠 6
news_df.loc[(news_df.Subject == '경제'), 'Subject'] = 0
news_df.loc[(news_df.Subject == '사회'), 'Subject'] = 1
news_df.loc[(news_df.Subject == '정치'), 'Subject'] = 2
news_df.loc[(news_df.Subject == '문화'), 'Subject'] = 3
news_df.loc[(news_df.Subject == '국제'), 'Subject'] = 4
news_df.loc[(news_df.Subject == 'IT'), 'Subject'] = 5
news_df.loc[(news_df.Subject == '스포츠'), 'Subject'] = 6
news_df = news_df.astype({'Subject' : int})

CPU times: user 268 ms, sys: 11.2 ms, total: 279 ms
Wall time: 280 ms


In [12]:
news_df.head()

,ID,Company,Year,Headline,Content,News,Subject
63013,20191103192315001,02100201,2019,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복,머니투데이 방콕태국김성휘 기자 경쟁력 높은 시장 진출 위한 교두보 주형철 청와대 ...,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복머니투데이 방콕태국김성휘 기자 ...,0
63014,20191103153210001,02100201,2019,사진코리아 세일 페스타 즐겨볼까,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중...,사진코리아 세일 페스타 즐겨볼까머니투데이 김휘선 기자 코리아세일페스타가 시작한 ...,0
63015,20191103120110001,02100201,2019,대기업 거래 신평사 지정 관행 부담,머니투데이 김지훈 기자 중기중앙회 대기업 자발적 거래관행 개선 촉구 자료중소기업중앙...,대기업 거래 신평사 지정 관행 부담머니투데이 김지훈 기자 중기중앙회 대기업 자발...,0
63016,20191103173816001,02100201,2019,년째 썩지않은 맥도날드 치즈버거 정말,머니투데이 김도엽 인턴 아이슬란드의 한 고객이 년 구매 현재도 썩지 않고 그대로 보...,년째 썩지않은 맥도날드 치즈버거 정말머니투데이 김도엽 인턴 아이슬란드의 한 고객이 ...,3
63018,20191103152648001,02100201,2019,사진첫 주말 코리아 세일 페스타 북적이는 명동,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중구...,사진첫 주말 코리아 세일 페스타 북적이는 명동머니투데이 김휘선 기자 코리아세일페스...,0


In [13]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161217 entries, 63013 to 256426
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   ID        161217 non-null  object
 1   Company   161217 non-null  object
 2   Year      161217 non-null  object
 3   Headline  161217 non-null  object
 4   Content   161217 non-null  object
 5   News      161217 non-null  object
 6   Subject   161217 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 9.8+ MB


In [14]:
%%time
news_df.to_csv('news_df_02100201.csv', index = False)

CPU times: user 6.49 s, sys: 445 ms, total: 6.94 s
Wall time: 7 s


In [15]:
%%time
data = pd.read_csv('news_df_02100201.csv')
data.head()

CPU times: user 5.62 s, sys: 499 ms, total: 6.12 s
Wall time: 6.2 s


,ID,Company,Year,Headline,Content,News,Subject
0,20191103192315001,2100201,2019,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복,머니투데이 방콕태국김성휘 기자 경쟁력 높은 시장 진출 위한 교두보 주형철 청와대 ...,한인도네시아 자유무역협정 월 타결아반떼 딜레마 극복머니투데이 방콕태국김성휘 기자 ...,0
1,20191103153210001,2100201,2019,사진코리아 세일 페스타 즐겨볼까,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중...,사진코리아 세일 페스타 즐겨볼까머니투데이 김휘선 기자 코리아세일페스타가 시작한 ...,0
2,20191103120110001,2100201,2019,대기업 거래 신평사 지정 관행 부담,머니투데이 김지훈 기자 중기중앙회 대기업 자발적 거래관행 개선 촉구 자료중소기업중앙...,대기업 거래 신평사 지정 관행 부담머니투데이 김지훈 기자 중기중앙회 대기업 자발...,0
3,20191103173816001,2100201,2019,년째 썩지않은 맥도날드 치즈버거 정말,머니투데이 김도엽 인턴 아이슬란드의 한 고객이 년 구매 현재도 썩지 않고 그대로 보...,년째 썩지않은 맥도날드 치즈버거 정말머니투데이 김도엽 인턴 아이슬란드의 한 고객이 ...,3
4,20191103152648001,2100201,2019,사진첫 주말 코리아 세일 페스타 북적이는 명동,머니투데이 김휘선 기자 코리아세일페스타가 시작한 후 첫 주말인 일 오후 서울 중구...,사진첫 주말 코리아 세일 페스타 북적이는 명동머니투데이 김휘선 기자 코리아세일페스...,0


In [38]:
num = int(len(data) * 0.7)
train_data = data[:num]
test_data = data[num:]
print(len(train_data))
print(len(test_data))

112851
48366


In [17]:
train_data['Subject'].value_counts()

0    39001
1    19038
2    17954
3    12535
4    12197
5    11039
6     1087
Name: Subject, dtype: int64

In [18]:
test_data['Subject'].value_counts()

0    16411
1     8475
2     7576
3     5314
4     5300
5     4850
6      440
Name: Subject, dtype: int64

## 토큰화

konlpy의 mecab을 이용하여 뉴스 데이터를 토큰화 시켜 피쳐로 사용할 수 있도록 합니다

In [19]:
mecab = Mecab()

In [20]:
%%time
X_train = []
for sentence in train_data['News']:
    temp_X = mecab.nouns(sentence)
    temp_X = [i for i in temp_X if len(i) > 1]
    X_train.append(temp_X)

CPU times: user 1min 50s, sys: 2.78 s, total: 1min 52s
Wall time: 1min 52s


In [24]:
%%time
df = pd.DataFrame(X_train)
df.to_csv('X_train.csv', index = None, header = None)

CPU times: user 1min 18s, sys: 4.12 s, total: 1min 22s
Wall time: 1min 23s


In [25]:
%%time
X_test = []
for sentence in test_data['News']:
    temp_X = mecab.nouns(sentence)
    temp_X = [i for i in temp_X if len(i) > 1]
    X_test.append(temp_X)

CPU times: user 50 s, sys: 515 ms, total: 50.5 s
Wall time: 51 s


In [30]:
test = data['News'][150000]
test_n = [mecab.nouns(test)]
test_n

[['현아',
  '란제리',
  '룩',
  '패턴',
  '슈트',
  '스타일',
  '투데이',
  '이은',
  '기자',
  '가수',
  '현아',
  '사진',
  '제공',
  '가수',
  '현아',
  '던',
  '커플',
  '패션',
  '화보',
  '속',
  '스타일',
  '패션',
  '매거진',
  '코리아',
  '월호',
  '커버',
  '모델',
  '현아',
  '던',
  '발탁',
  '이',
  '화보',
  '공개',
  '앤',
  '리치',
  '콘셉트',
  '진행',
  '이번',
  '화보',
  '현아',
  '던',
  '자동차',
  '브랜드',
  '아우디',
  '뉴',
  '아우디',
  '뉴',
  '아우디',
  '스타일리시',
  '드라이빙',
  '모습',
  '화보',
  '속',
  '현아',
  '패턴',
  '멋',
  '란제리',
  '컬러',
  '배색',
  '멋',
  '핑크',
  '빛',
  '재킷',
  '파격',
  '룩',
  '완성',
  '여기',
  '현아',
  '버건',
  '디',
  '색',
  '롱',
  '부츠',
  '매치',
  '눈길',
  '팬',
  '턴',
  '패턴',
  '슈트',
  '셔츠',
  '매혹',
  '스타일',
  '연출',
  '가수',
  '현아',
  '사진',
  '제공',
  '현아',
  '던',
  '아우디',
  '자동차',
  '배경',
  '매혹',
  '분위기',
  '커플',
  '화보',
  '다리',
  '라인',
  '레이스',
  '드레스',
  '현아',
  '화이트',
  '슈트',
  '던',
  '포옹',
  '듯',
  '자연',
  '포즈',
  '눈길',
  '뉴',
  '아우디',
  '에서',
  '포즈',
  '가수',
  '사진',
  '제공',
  '코리아',
  '화보',
  '속',
  '던',
  '크리스탈',
  '재킷',
  '청바지',
  '매치',
  '

## 정수 인코딩

토큰화 된 단어를 정수로 인코딩합니다

In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [32]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 106099
등장 빈도가 2번 이하인 희귀 단어의 수: 39420
단어 집합에서 희귀 단어의 비율: 37.15397883109171
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.2765843954972957


In [33]:
# 전체 단어 개수 중 빈도수 5이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 66681


In [34]:
%%time
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

CPU times: user 19.5 s, sys: 946 ms, total: 20.5 s
Wall time: 20.7 s


In [35]:
y_train = np.array(train_data['Subject'])
y_test = np.array(test_data['Subject'])

## 빈 샘플 제거

In [39]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [40]:
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

112851
112851


/Users/donghyeon/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


## 패딩

정수로 인코딩된 토큰화 

In [41]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 2751
리뷰의 평균 길이 : 167.20370222683007


In [42]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [43]:
max_len = 500
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 500 이하인 샘플의 비율: 98.33320041470611


In [44]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [45]:
y_train = to_categorical(y_train) # 훈련용 뉴스 기사 레이블의 원-핫 인코딩
y_test = to_categorical(y_test) # 테스트용 뉴스 기사 레이블의 원-핫 인코딩

## 딥러닝 모델 생성

분류를 위해 딥러닝 모델을 생성합니다

In [47]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(7, activation='softmax'))

In [48]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1,
                     save_best_only=True)

In [49]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [50]:
history = model.fit(X_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc],
                    validation_data=(X_test, y_test))

Epoch 1/30
882/882 [==============================] - 753s 852ms/step - loss: 1.0517 - acc: 0.6351 - val_loss: 0.6356 - val_acc: 0.7981

Epoch 00001: val_acc improved from -inf to 0.79810, saving model to best_model.h5
Epoch 2/30
882/882 [==============================] - 746s 846ms/step - loss: 0.5254 - acc: 0.8356 - val_loss: 0.5768 - val_acc: 0.8102

Epoch 00002: val_acc improved from 0.79810 to 0.81018, saving model to best_model.h5
Epoch 3/30
882/882 [==============================] - 743s 843ms/step - loss: 0.3664 - acc: 0.8790 - val_loss: 0.6201 - val_acc: 0.8039

Epoch 00003: val_acc did not improve from 0.81018
Epoch 4/30
882/882 [==============================] - 744s 843ms/step - loss: 0.3060 - acc: 0.9000 - val_loss: 0.6060 - val_acc: 0.8103

Epoch 00004: val_acc improved from 0.81018 to 0.81026, saving model to best_model.h5
Epoch 5/30
882/882 [==============================] - 736s 834ms/step - loss: 0.2350 - acc: 0.9232 - val_loss: 0.6321 - val_acc: 0.8059

Epoch 00005: 

In [51]:
model.save('best_model.h5')

In [52]:
loaded_model = load_model('best_model.h5')

In [53]:
loaded_model.evaluate(X_test, y_test)[1]

1512/1512 [==============================] - 81s 54ms/step - loss: 0.6560 - acc: 0.8061


0.8061448335647583

In [54]:
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

1512/1512 [==============================] - 80s 53ms/step - loss: 0.6560 - acc: 0.8061

 테스트 정확도: 0.8061


In [58]:
predict = loaded_model.predict(X_test)

In [59]:
predict[0]

array([2.2367476e-02, 9.4359778e-02, 8.7769860e-01, 1.1295389e-03,
       3.3867511e-03, 9.6512225e-04, 9.2726776e-05], dtype=float32)

In [60]:
np.argmax(predict[0])

2